In [1]:
import cv2
import os
import numpy as np
from tensorflow.keras.models import load_model
import time

In [2]:
print(cv2.__version__)

4.0.1


In [3]:
model=load_model('../Model/model_mobilenetv2.h5')

In [4]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
flatten (Flatten)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               327936    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)               

In [5]:
model_file = "../Model/res10_300x300_ssd_iter_140000.caffemodel"
config_file = "../Model/deploy.prototxt"
dnn_detector = cv2.dnn.readNetFromCaffe(config_file, model_file)

In [6]:
vid = cv2.VideoCapture(0)
while(True):
    #time.sleep(1)
    ret, frame = vid.read()

    h, w = frame.shape[:2]
    
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 117.0, 123.0))
    dnn_detector.setInput(blob)
    faces = dnn_detector.forward()
    
    for i in range(faces.shape[2]):
        confidence = faces[0, 0, i, 2]
        if confidence > 0.5:
            
            box = faces[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x, y, x1, y1) = box.astype("int")
            
            crop_img = frame[y:y1, x:x1]
            crop_img = cv2.resize(crop_img, (224, 224))
            crop_img = crop_img.reshape([1, 224, 224, 3])
            crop_img = crop_img / 255
            
            pred = model.predict(crop_img)
            
            if pred[0, 1] > 0.95:
                cv2.putText(frame, "No Mask : " + str(np.round(pred[0, 1], 2) * 100) + str("%"), (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
                cv2.rectangle(frame, (x, y), (x1, y1), (0, 0, 255), 2)
                print("No Mask", pred)
            elif (pred[0, 1] > 0.01) or (pred[0, 0] < 0.999):
                cv2.putText(frame, "Wear properly : " + str(np.round(pred[0, 0], 2) * 100) + str("%"), (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 0, 255), 2)
                cv2.rectangle(frame, (x, y), (x1, y1), (255, 0, 255), 2)
                #print("no Mask  :  ", pred[0, 0] )
                print("Wear properly", pred)
            else:
                cv2.putText(frame, "Mask : " + str(np.round(pred[0, 0], 2) * 100) + str("%"), (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)
                cv2.rectangle(frame, (x, y), (x1, y1), (0, 255, 0), 2)
                #print("no Mask  :  ", pred[0, 0] )
                print("Mask", pred)
             
    cv2.imshow('frame', frame)
    
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord('q'):
        break

vid.release()
cv2.destroyAllWindows()

Wear properly [[0.9952127  0.00478729]]
Wear properly [[0.98671025 0.01328969]]
Wear properly [[0.9884723  0.01152769]]
Mask [[9.991936e-01 8.063390e-04]]
Wear properly [[0.9877943 0.0122057]]
Wear properly [[0.9969146  0.00308541]]
Wear properly [[0.9935368  0.00646316]]
Wear properly [[0.99791473 0.00208528]]
Wear properly [[0.9938498  0.00615025]]
Mask [[9.9964797e-01 3.5204185e-04]]
Wear properly [[0.9617378  0.03826216]]
Wear properly [[0.98340374 0.01659623]]
Wear properly [[0.968388   0.03161198]]
Wear properly [[0.98454154 0.01545851]]
Wear properly [[0.98722523 0.0127747 ]]
Wear properly [[0.9921131 0.0078869]]
Wear properly [[0.98155725 0.01844277]]
Wear properly [[0.99469924 0.00530074]]
Wear properly [[0.9136665  0.08633351]]
Wear properly [[0.93872666 0.06127332]]
Wear properly [[0.93870795 0.06129202]]
Wear properly [[0.9448398  0.05516023]]
Wear properly [[0.6989274  0.30107257]]
Wear properly [[0.956319   0.04368106]]
Wear properly [[0.97283494 0.02716505]]
Wear properl

No Mask [[4.0829932e-06 9.9999595e-01]]
No Mask [[1.7616564e-06 9.9999821e-01]]
No Mask [[2.6720687e-07 9.9999976e-01]]
No Mask [[4.369417e-06 9.999956e-01]]
No Mask [[0.00177476 0.9982253 ]]
Wear properly [[0.99236476 0.00763527]]
Wear properly [[0.9882648  0.01173524]]
Mask [[9.999293e-01 7.069810e-05]]
Mask [[9.999316e-01 6.847033e-05]]
Mask [[9.9997294e-01 2.7062693e-05]]
Mask [[9.999701e-01 2.989936e-05]]
Mask [[9.9999762e-01 2.4302083e-06]]
Mask [[9.9999499e-01 4.9778837e-06]]
Mask [[9.9999857e-01 1.4611617e-06]]
Mask [[9.999875e-01 1.254901e-05]]
Mask [[9.9998963e-01 1.0376447e-05]]
Mask [[9.9998009e-01 1.9858633e-05]]
Mask [[9.9999785e-01 2.1814749e-06]]
Mask [[9.999093e-01 9.073679e-05]]
Mask [[9.9999332e-01 6.7083984e-06]]
Mask [[9.9994934e-01 5.0639501e-05]]
Mask [[9.9988210e-01 1.1782972e-04]]
Mask [[9.9998438e-01 1.5600179e-05]]
Mask [[9.999943e-01 5.672700e-06]]
Mask [[9.9999738e-01 2.6393018e-06]]
Mask [[9.999900e-01 9.967892e-06]]
Mask [[9.9981123e-01 1.8873968e-04]]
We

Wear properly [[0.96386844 0.03613153]]
Wear properly [[0.8771154  0.12288462]]
Wear properly [[0.9942993  0.00570069]]
Wear properly [[0.99579465 0.00420541]]
Wear properly [[0.9551501  0.04484982]]
Wear properly [[0.9893811  0.01061889]]
Wear properly [[0.9894047  0.01059536]]
Wear properly [[0.9947865  0.00521348]]
Wear properly [[0.994565   0.00543498]]
Wear properly [[0.99703753 0.00296249]]
Wear properly [[0.99712235 0.00287762]]
Wear properly [[0.95395845 0.04604156]]
Wear properly [[0.9782597  0.02174038]]
Wear properly [[0.98233    0.01767005]]
Wear properly [[0.46788388 0.5321162 ]]
Wear properly [[0.96187675 0.03812322]]
Wear properly [[0.7262792  0.27372086]]
Wear properly [[0.9421752  0.05782482]]
Wear properly [[0.82052696 0.1794731 ]]
Wear properly [[0.4628971 0.5371029]]
Wear properly [[0.53361714 0.4663829 ]]
Wear properly [[0.4882715  0.51172847]]
Wear properly [[0.5000097  0.49999028]]
Wear properly [[0.877221   0.12277909]]
Wear properly [[0.8504114  0.14958853]]
We

Wear properly [[0.9953289  0.00467111]]
Wear properly [[0.98980194 0.01019808]]
Wear properly [[0.95602643 0.04397352]]
Wear properly [[0.99128824 0.00871178]]
Wear properly [[0.9830034  0.01699654]]
Wear properly [[0.97949654 0.02050345]]
Wear properly [[0.9460331  0.05396692]]
Wear properly [[0.99875045 0.00124951]]
Wear properly [[0.9846087  0.01539135]]
Wear properly [[0.9931919  0.00680812]]
Wear properly [[0.97318745 0.02681255]]
Wear properly [[0.9821744  0.01782567]]
Wear properly [[0.98788494 0.01211503]]
Wear properly [[0.9699159  0.03008408]]
Wear properly [[0.9475608  0.05243923]]
Wear properly [[0.9913948  0.00860514]]
Wear properly [[0.9945029  0.00549711]]
Wear properly [[0.9365387  0.06346127]]
Wear properly [[0.98174703 0.01825305]]
Wear properly [[0.9577987  0.04220128]]
Wear properly [[0.9905652  0.00943479]]
Wear properly [[0.9078078  0.09219217]]
Wear properly [[0.9957801  0.00421983]]
Wear properly [[0.9916467  0.00835336]]
Wear properly [[0.9274785  0.07252154]]


Wear properly [[0.93492633 0.06507362]]
Wear properly [[0.9853158  0.01468421]]
Wear properly [[0.96202534 0.0379747 ]]
Wear properly [[0.9752161  0.02478395]]
Wear properly [[0.8748369 0.1251631]]
Wear properly [[0.88852435 0.11147565]]
Wear properly [[0.9448877  0.05511229]]
Wear properly [[0.9713685  0.02863156]]
Wear properly [[0.971331   0.02866898]]
Wear properly [[0.8624685  0.13753156]]
Wear properly [[0.9619735  0.03802649]]
Wear properly [[0.91989803 0.08010195]]
Wear properly [[0.8459386  0.15406138]]
Wear properly [[0.9402016  0.05979843]]
Wear properly [[0.5085427  0.49145728]]
Wear properly [[0.77190757 0.22809248]]
Wear properly [[0.20326762 0.79673237]]
Wear properly [[0.80692196 0.19307806]]
No Mask [[2.7748890e-06 9.9999726e-01]]
No Mask [[4.9741453e-07 9.9999952e-01]]
No Mask [[1.2179474e-05 9.9998784e-01]]
No Mask [[1.2919584e-06 9.9999869e-01]]
No Mask [[1.7487407e-06 9.9999821e-01]]
No Mask [[6.763674e-05 9.999324e-01]]
No Mask [[5.2931150e-06 9.9999475e-01]]
No M